In [0]:
import pickle
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, make_scorer

To make a Bland we will need predictions from the previous step, which we saved, let's load them

In [0]:
with open('/content/drive/My Drive/test_task/review_preds.pickle', 'rb') as f:
    review_preds = pickle.load(f)
with open('/content/drive/My Drive/test_task/summury_preds.pickle', 'rb') as f:
    summury_preds = pickle.load(f)
with open('/content/drive/My Drive/test_task/target.pickle', 'rb') as f:
    target = pickle.load(f)

Concatenate predictions of two Berts:

In [0]:
features = np.hstack((review_preds,summury_preds))

In [0]:
target = np.array(target)

I would like to choose KNN as it can fit well on difficult data (don't think that difference of predictions of Berts will be big, but anyway, let's use it)  

In [0]:
neigh = KNeighborsClassifier(n_neighbors=35)

f1 = make_scorer(f1_score, average='micro')

Firstly, let's check the model on cross-validation:


In [0]:
res = cross_validate(neigh, features, target, scoring=f1, n_jobs=-1, cv=3, return_train_score=True, verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.8s finished


In [0]:
print('Test F1: % 0.4f \nTrain F1: % 0.4f ' % (np.mean(res['test_score']), 
                                               np.mean(res['train_score'])))

Test F1:  0.7716 
Train F1:  0.7804 


As I expected this model will get better performance, than two Bert's separately. Also, I want to note that I choose a number of neighbors - 35 that can make our model more robust to overfitting.

In [0]:
neigh.fit(features, target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=35, p=2,
                     weights='uniform')

Saving model to evaluate in future:

In [0]:
with open('/content/drive/My Drive/test_task/neigh.pickle', 'wb') as f:
    pickle.dump(neigh, f)

Also, I would like to have a look at performance, if we would just average predictions that we have.

In [0]:
simple_vote = np.argmax((review_preds + summury_preds), axis = 1)

In [0]:
f1_score(target, simple_vote, average='micro')

0.76605

Obviously looks worse then KNN's